In [2]:
!pip install autogluon
!git clone https://github.com/priorlabs/tabpfn-extensions.git
!pip install -e tabpfn-extensions
!pip install tabpfn
!pip install tabpfn_extensions

fatal: destination path 'tabpfn-extensions' already exists and is not an empty directory.
Obtaining file:///content/tabpfn-extensions
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 155.5/155.5 kB 5.4 MB/s eta 0:00:00
  Building editable for tabpfn-extensions (pyproject.toml) ... done
  Created wheel for tabpfn-extensions: filename=tabpfn_extensions-0.1.1-0.editable-py3-none-any.whl size=13301 sha256=25e148a2ae71becb38677223e4243f1c0a662ff02de1ddf0ef4f182432eaa197
  Stored in directory: /tmp/pip-ephem-wheel-cache-q8ce0h7w/wheels/8f/b7/2c/ec18b502697068b39e1503426a50b729bdac046bed4810759d
Successfully built tabpfn-extensions


In [3]:
# prompt: import pandas and basic machine learning models for regression

import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR


from sklearn.model_selection import train_test_split

import itertools
import random

import torch
import random
import numpy as np

import os
import joblib

import matplotlib.pyplot as plt

from tabpfn import TabPFNRegressor
from sklearn.model_selection import KFold
from sklearn.multioutput import MultiOutputRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import PolynomialFeatures

from sklearn.metrics import mean_absolute_percentage_error

from sklearn.linear_model import LinearRegression

# from sklearn.metrics import mean_absolute_percentage_error
# from tabpfn_extensions.post_hoc_ensembles.sklearn_interface import AutoTabPFNRegressor

In [4]:
from itertools import combinations
from scipy.special import comb


In [5]:
from tabpfn.model.loading import (
    load_fitted_tabpfn_model,
    save_fitted_tabpfn_model,
)

In [8]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [9]:
class EagleBlendPredictor:
    def __init__(self, model_sources):
        """
        model_sources: Dict[str, Any]
            A dictionary where keys are 'BlendProperty1', ..., 'BlendProperty10'
            and values are:
              - loaded model objects, or
              - callables returning models, or
              - custom loading logic (you will supply these)
        """
        self.home = '/content/drive/MyDrive/Projects/Shell/App/Models'
        self.saved_files_map = {
                      1: {
                          "model": 'linear_model_poly_target_1.joblib',
                          "transform": 'poly1_features.joblib'
                      },
                      2: {
                          "model": 'linear_model_poly_target_2.joblib',
                          "transform": 'poly2_features.joblib'
                      },
                      5: {
                          "model": 'tabpfn_model_target_5.tabpfn_fit', #tabpfn_model_target_5_cpu.tabpfn_fit,'tabpfn_model_target_5_cpu.tabpfn_fit'
                          "transform": 'poly5_features.joblib'
                      },
                      6: {
                          "model": 'linear_model_poly_target_6.joblib',
                          "transform": 'poly6_features.joblib'
                      },
                      7: {
                          "model": 'tabpfn_model_target_7.tabpfn_fit',
                          # For Property 7, the transformation is the mixture feature generation,
                          # which is not a saved object like PolynomialFeatures.
                          # You would need to apply the generate_mixture_features function.
                          "transform_function": "generate_mixture_features"
                      },
                      8: {
                          # For Property 8, the "model" is the initial prediction model (not explicitly saved in this workflow)
                          # and the correction is the piecewise function defined by parameters and threshold.
                          "params": 'piecewise_params_prop8.joblib',
                          "threshold": 'piecewise_threshold_prop8.joblib',
                          "correction_function": "piecewise_model" # Reference the function name
                      },
                      10: {
                          "model": 'linear_model_poly_target_10.joblib',
                          "transform": 'poly10_features.joblib'
                      }
                  }


        self.models = {}
        # Load models and transformers manually
        self.model_1 = joblib.load(os.path.join(self.home, self.saved_files_map[1]["model"]))
        self.poly_1 = joblib.load(os.path.join(self.home, self.saved_files_map[1]["transform"]))

        self.model_2 = joblib.load(os.path.join(self.home, self.saved_files_map[2]["model"]))
        self.poly_2 = joblib.load(os.path.join(self.home, self.saved_files_map[2]["transform"]))

        self.model_5 = load_fitted_tabpfn_model(
            os.path.join(self.home, self.saved_files_map[5]["model"]), #device="cpu"
        )
        self.poly_5 = joblib.load(os.path.join(self.home, self.saved_files_map[5]["transform"]))

        self.model_6 = joblib.load(os.path.join(self.home, self.saved_files_map[6]["model"]))
        self.poly_6 = joblib.load(os.path.join(self.home, self.saved_files_map[6]["transform"]))

        self.model_7 = load_fitted_tabpfn_model(
            os.path.join(self.home, self.saved_files_map[7]["model"]), #device="cpu"
        )
        # No saved transform for model_7 — use generate_mixture_features later in prediction
        self.piecewise_params_8 = joblib.load(os.path.join(self.home, self.saved_files_map[8]["params"]))
        self.piecewise_threshold_8 = joblib.load(os.path.join(self.home, self.saved_files_map[8]["threshold"]))

        # Use piecewise_model function later

        self.model_10 = joblib.load(os.path.join(self.home, self.saved_files_map[10]["model"]))
        self.poly_10 = joblib.load(os.path.join(self.home, self.saved_files_map[10]["transform"]))
        pass


    def piecewise_model(self, x, boundaries=np.linspace(-0.2, 0.2, 10+1)[1:-1]):
        """
        x: a single float value
        params: list of 20 parameters [A1, B1, A2, B2, ..., A10, B10]
        boundaries: 9 values that divide x into 10 regions
        """
        params = self.piecewise_params_8
        # Unpack parameters
        segments = [(params[i], params[i+1]) for i in range(0, 20, 2)]

        # Piecewise logic using boundaries
        for i, bound in enumerate(boundaries):
            if x < bound:
                A, B = segments[i]
                return A * x + B
        # If x is greater than all boundaries, use the last segment
        A, B = segments[-1]
        return A * x + B

    def predict_BlendProperty1(self, row, full = False):
        # Dummy custom transformation and prediction for BlendProperty1
        if full:
            features = self._transform1(row)
        else:
            features = self.poly_1.transform(row)
        return self.model_1.predict(features)


    def predict_BlendProperty2(self, row, full = False):
        if full:
            pass
            features = self._transform2(row)
            features = self.poly_2.transform(features)
        else:
            features = self.poly_2.transform(row)

        return self.model_2.predict(features)


    def predict_BlendProperty3(self, row,full = False):
        pass
        # model = self.models["BlendProperty3"]
        # features = self._transform3(row)
        return model.predict(features)



    def predict_BlendProperty4(self, row, full = False):
        pass
        model = self.models["BlendProperty4"]
        features = self._transform4(row)
        return model.predict(features)


    def predict_BlendProperty5(self, row, full =False ):
        if full:
            features = self._transform5(row)
        else:
            features = self.poly_5.transform(row)

        return self.model_5.predict(features)

    def predict_BlendProperty6(self, row, full=False):
        if full:
            features = self._transform6(row)
        else:
            features = self.poly_6.transform(row)
        return self.model_6.predict(features)



    def predict_BlendProperty7(self, row, full =False):
        if full:

            features = self._transform7(row)
        else:
            pass

        return self.model_7.predict(features)



    def predict_BlendProperty8(self, row,full = False):
        first_predict_Res = 2* random.random() -1 ## first prediction
        if abs(first_predict_Res) < self.threshold:
            first_predict_Res = self.piecewise_model(first_predict_Res)

        # features = self._transform8(row)
        return first_predict_Res

    def predict_BlendProperty9(self, row, full = False):
        pass

    def predict_BlendProperty10(self, row, full = False):
        if full:
            pass
            features = self._transform10(row)
        else:
            features = self.poly_10.transform(row)

        return self.model_10.predict(features)

    # Dummy transformation functions (replace with your actual logic later)
    def _transform1(self, row): return None
    def _transform2(self, row): return None
    def _transform3(self, row): return None
    def _transform4(self, row): return None
    def _transform5(self, row): return None
    def _transform6(self, row): return None

    # def _transform7(self, row): return self._to_feature_array(row)
    def _transform7(self, df: pd.DataFrame) -> pd.DataFrame:
        tn = 7
        fn = tn

        property_tn = [f'Component{i+1}_Property{fn}' for i in range(5)]
        fraction_cols = [f'Component{i+1}_fraction' for i in range(5)]
        target_cols = [f"BlendProperty{i}" for i in range(1, 11)]

        # Generate mixture features using the class method
        df_prop7 = df[fraction_cols + property_tn]
        mixture_features = self.generate_mixture_features(df_prop7)

        # Identify columns to concatenate (exclude targets and fraction/property 7 columns)
        exclude_cols = target_cols + fraction_cols + property_tn
        other_features = df[[col for col in df.columns if col not in exclude_cols]]

        # Reset indices and concatenate
        mixture_features = mixture_features.reset_index(drop=True)
        other_features = other_features.reset_index(drop=True)

        return pd.concat([mixture_features, other_features], axis=1)



    def _transform8(self, row): return None
    def _transform9(self, row): return None
    def _transform10(self, row): return None



    def generate_mixture_features(data):
        """
        Generate symmetric and weighted nonlinear interactions between fuel weights and properties.
        The input 'data' should contain weights in the first 5 columns/elements and properties in the next 5.

        :param data: np.ndarray, pd.DataFrame, or list of shape (n_samples, 10) or (10,)
        :return: pd.DataFrame with generated features.
        """
        # Convert input to numpy array and handle single row/list input
        if isinstance(data, pd.DataFrame):
            data_array = data.values
        elif isinstance(data, list):
            data_array = np.array(data)
        elif isinstance(data, np.ndarray):
            data_array = data
        else:
            raise TypeError("Input data must be a pandas DataFrame, numpy array, or list.")

        # Reshape single row/list input to 2D array
        if data_array.ndim == 1:
            data_array = data_array.reshape(1, -1)

        # Ensure the input has 10 columns (5 weights + 5 properties)
        if data_array.shape[1] != 10:
            raise ValueError("Input data must have 10 columns/elements (5 weights and 5 properties).")

        # Separate weights and properties
        W = data_array[:, :5]
        P = data_array[:, 5:]


        n_samples, n_fuels = W.shape
        features = {}

        # Original weights and properties
        for i in range(n_fuels):
            features[f'w{i+1}'] = W[:, i]
            features[f'p{i+1}'] = P[:, i]
            features[f'w{i+1}_p{i+1}'] = W[:, i] * P[:, i]  # weighted property

        # --- 1. Weighted sum of properties ---
        features['weighted_sum'] = np.sum(W * P, axis=1)

        # --- 2. Weighted square of properties ---
        features['weighted_sum_sq'] = np.sum(W * P**2, axis=1)

        # --- 3. Weighted tanh of properties ---
        features['weighted_tanh'] = np.sum(W * np.tanh(P), axis=1)

        # --- 4. Weighted exponential ---
        # features['weighted_exp'] = np.sum(W * np.exp(P), axis=1)
        # Clip P before exponential to avoid overflow
        safe_exp = np.exp(np.clip(P, a_min=None, a_max=50))  # 50 is safe upper bound
        features['weighted_exp'] = np.sum(W * safe_exp, axis=1)


        # --- 5. Weighted logarithm (clip to avoid -inf) ---
        # features['weighted_log'] = np.sum(W * np.log(np.clip(P, 1e-6, None)), axis=1)
        features['weighted_log'] = np.sum(W * np.log(np.clip(P, 1e-6, None)), axis=1)


        # --- 6. Pairwise interactions (symmetric, weighted) ---
        for i, j in combinations(range(n_fuels), 2):
            pij = P[:, i] * P[:, j]
            wij = W[:, i] * W[:, j]
            features[f'pair_p{i+1}p{j+1}'] = pij
            features[f'weighted_pair_p{i+1}p{j+1}'] = pij * wij

        # --- 7. Triple interactions (weighted & symmetric) ---
        for i, j, k in combinations(range(n_fuels), 3):
            pij = P[:, i] * P[:, j] * P[:, k]
            wij = W[:, i] * W[:, j] * W[:, k]
            features[f'triplet_p{i+1}{j+1}{k+1}'] = pij
            features[f'weighted_triplet_p{i+1}{j+1}{k+1}'] = pij * wij

        # --- 8. Power series + weight modulated ---
        for power in [2, 3, 4]:
            features[f'power_sum_{power}'] = np.sum(W * P**power, axis=1)

        # --- 9. Log-weighted property (prevent log(0)) ---
        logW = np.log(np.clip(W, 1e-6, None))
        features['log_weighted_p'] = np.sum(logW * P, axis=1)

        # --- 10. Symmetric polynomial combinations (elementary symmetric) ---
        # Up to degree 5 (since you have 5 fuels)
        for r in range(1, 6):
            key = f'e_sym_poly_r{r}'
            val = np.zeros(n_samples)
            for idx in combinations(range(n_fuels), r):
                prod_p = np.prod(P[:, idx], axis=1)
                val += prod_p
            features[key] = val

        # --- 11. Weighted interaction difference (symmetry in differences) ---
        for i, j in combinations(range(n_fuels), 2):
            diff = P[:, i] - P[:, j]
            wdiff = W[:, i] * W[:, j]
            features[f'weighted_diff_p{i+1}{j+1}'] = diff * wdiff

        # --- 12. Mean, max, min (weighted) ---
        total_weight = np.sum(W, axis=1, keepdims=True)
        weighted_mean = np.sum(W * P, axis=1) / np.clip(total_weight.squeeze(), 1e-6, None)
        features['weighted_mean'] = weighted_mean
        features['max_prop'] = np.max(P, axis=1)
        features['min_prop'] = np.min(P, axis=1)

        # --- 13. Weighted cross-log terms ---
        for i, j in combinations(range(n_fuels), 2):
            log_mix = np.log(np.clip(P[:, i] + P[:, j], 1e-6, None))
            wij = W[:, i] * W[:, j]
            features[f'logsum_p{i+1}{j+1}'] = log_mix * wij

        # --- 14. Inverse + weighted inverse ---
        # features['inv_prop_sum'] = np.sum(W / np.clip(P, 1e-6, None), axis=1)
        features['inv_prop_sum'] = np.sum(W / np.clip(P, 1e-6, None), axis=1)


        # --- 15. Weighted relu (max(p, 0)) ---
        relu = np.maximum(P, 0)
        features['weighted_relu'] = np.sum(W * relu, axis=1)

        # --- 16. Weighted sin/cos transforms ---
        features['weighted_sin'] = np.sum(W * np.sin(P), axis=1)
        features['weighted_cos'] = np.sum(W * np.cos(P), axis=1)

        # --- 17. Normalized properties ---
        prop_sum = np.sum(P, axis=1, keepdims=True)
        normalized_P = P / np.clip(prop_sum, 1e-6, None)
        for i in range(n_fuels):
            features[f'norm_p{i+1}'] = normalized_P[:, i]

        # --- 18. Product of all p's and all w's ---
        features['total_product_p'] = np.prod(P, axis=1)
        features['total_product_w'] = np.prod(W, axis=1)

        # --- 19. Mixed entropic form ---
        # entropy_like = -np.sum(W * np.log(np.clip(W, 1e-6, None)), axis=1)
        # features['entropy_weights'] = entropy_like

        # Convert to DataFrame
        df = pd.DataFrame(features)

        return df

In [ ]:
tn = 5
fn = tn

property_tn = [f'Component{i+1}_Property{fn}' for i in range(5)]
fraction_cols = [f'Component{i+1}_fraction' for i in range(5)]
target_cols = [f"BlendProperty{i}" for i in range(1, 11)]
components = ['Component1', 'Component2', 'Component3', 'Component4', 'Component5']
properties = [f'Property{i}' for i in range(1, 11)]
target = target_cols[tn-1]




### Plan Step 2: Update `_transform` methods for Column Selection

**Objective:** Modify each `_transform` method (`_transform1`, `_transform2`, etc.) to correctly identify and select the input features required by its corresponding model, regardless of whether the input `data` is a DataFrame containing many columns (like a full dataset) or just the specific columns needed for that property's prediction.

**Details:**

*   Each `_transform` method needs to know which columns from the input `data` are relevant for its specific property's model.
*   We can achieve this by:
    *   Storing the required input column names for each property, perhaps in a dictionary within the `__init__` method or as a separate configuration.
    *   Inside each `_transformX` method, check the type of the input `data`.
    *   If `data` is a pandas DataFrame, select the required columns using the stored column names.
    *   If `data` is a NumPy array or list, assume the columns are already in the correct order and pass it to `_to_feature_array` for reshaping. (This assumes that if a user passes a NumPy array or list, they are already providing only the necessary features in the correct order for that specific property's model).
    *   For transformations like `_transform7` that involve custom feature generation (`generate_mixture_features`), ensure that the necessary source columns (like component fractions and property 7 values) are correctly extracted before generating the mixture features.
*   The output of each `_transform` method should be in a format expected by the model's `predict` method (typically a 2D NumPy array after being processed by `_to_feature_array` or a transformer like `poly_X`).

**Implementation Notes:**

In [10]:
import torch
import os

# Define the path to the model file for target 7
home_dir = '/content/drive/MyDrive/Projects/Shell/App/Models'
model_filename_7 = 'tabpfn_model_target_5_cpu.tabpfn_fit'#'tabpfn_model_target_7.tabpfn_fit'
model_path_7 = os.path.join(home_dir, model_filename_7)

print(f"Attempting to load model directly using torch.load from: {model_path_7}")

try:
    # Load the model directly using torch.load with map_location
    # The file might contain a dictionary or the model state_dict
    # We need to inspect the file structure if this fails
    loaded_object = torch.load(model_path_7, map_location=torch.device('cpu'))

    # Assuming the saved file contains the model's state_dict or the model object itself
    # You might need to adjust this part based on how the model was saved
    # If loaded_object is the state_dict, you would need to instantiate the model
    # and then load the state_dict:
    # model_instance = YourTabPFNModelClass(...) # Instantiate your model
    # model_instance.load_state_dict(loaded_object)
    # loaded_model = model_instance
    # For now, let's assume loaded_object *is* the model object or something loadable

    # If the saved file contains the full model object:
    model_7_loaded_direct = loaded_object
    print("TabPFN model for Target 7 loaded successfully on CPU using torch.load.")

    # You can add a check here to see the type of the loaded object
    print(f"Type of loaded object: {type(model_7_loaded_direct)}")


except FileNotFoundError:
    print(f"Error: Model file not found at {model_path_7}. Please check the path.")
except Exception as e:
    print(f"An error occurred during direct model loading: {e}")

Attempting to load model directly using torch.load from: /content/drive/MyDrive/Projects/Shell/App/Models/tabpfn_model_target_5_cpu.tabpfn_fit
An error occurred during direct model loading: [enforce fail at inline_container.cc:175] . file in archive is not in a subdirectory: executor_state.joblib


In [8]:
from tabpfn.model.loading import load_fitted_tabpfn_model
import os

# Define the path to the model file for target 7
home_dir = '/content/drive/MyDrive/Projects/Shell/App/Models'
model_filename_7 = 'tabpfn_model_target_7.tabpfn_fit'
model_path_7 = os.path.join(home_dir, model_filename_7)

print(f"Attempting to load model using load_fitted_tabpfn_model from: {model_path_7}")

try:
    # Load the model using load_fitted_tabpfn_model
    model_7_loaded = load_fitted_tabpfn_model(model_path_7, device="cpu")
    print("TabPFN model for Target 7 loaded successfully on CPU using load_fitted_tabpfn_model.")

    # You can add a check here to see the type of the loaded object
    print(f"Type of loaded object: {type(model_7_loaded)}")


except FileNotFoundError:
    print(f"Error: Model file not found at {model_path_7}. Please check the path.")
except Exception as e:
    print(f"An error occurred during model loading: {e}")

Attempting to load model using load_fitted_tabpfn_model from: /content/drive/MyDrive/Projects/Shell/App/Models/tabpfn_model_target_7.tabpfn_fit


tabpfn-v2-regressor.ckpt:   0%|          | 0.00/44.4M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/37.0 [00:00<?, ?B/s]

TabPFN model for Target 7 loaded successfully on CPU using load_fitted_tabpfn_model.
Type of loaded object: <class 'tabpfn.regressor.TabPFNRegressor'>


## Test 5

In [8]:
import time

In [11]:
# Initiate the EagleBlendPredictor class
predictor = EagleBlendPredictor(model_sources={})
print("EagleBlendPredictor initiated successfully.")

RuntimeError: Attempting to deserialize object on a CUDA device but torch.cuda.is_available() is False. If you are running on a CPU-only machine, please use torch.load with map_location=torch.device('cpu') to map your storages to the CPU.

In [ ]:
# Define the path to the training data CSV
home_dir = '/content/drive/MyDrive/Projects/Shell/App/Data' # Assuming the CSV is in the same directory as models
train_csv_path = os.path.join(home_dir, 'train.csv') # Assuming the filename is train.csv

# Read the training data into a DataFrame
try:
    df = pd.read_csv(train_csv_path)
    df_train, df_test = train_test_split( df, test_size=0.2, random_state=42)
    print(f"Training data loaded successfully from {train_csv_path}")
    display(df_train.head())
except FileNotFoundError:
    print(f"Error: Training data file not found at {train_csv_path}. Please check the path.")
except Exception as e:
    print(f"An error occurred while loading training data: {e}")


# Select attributes for Property 1
tn = 5
fn = tn

predictors = [
    predictor.predict_BlendProperty1, predictor.predict_BlendProperty2, predictor.predict_BlendProperty3,
    predictor.predict_BlendProperty4, predictor.predict_BlendProperty5,predictor.predict_BlendProperty6,
    predictor.predict_BlendProperty7, predictor.predict_BlendProperty8, predictor.predict_BlendProperty9, predictor.predict_BlendProperty10
]

property_tn = [f'Component{i+1}_Property{fn}' for i in range(5)]
fraction_cols = [f'Component{i+1}_fraction' for i in range(5)]
target_cols = [f"BlendProperty{i}" for i in range(1, 11)]
components = ['Component1', 'Component2', 'Component3', 'Component4', 'Component5']
properties = [f'Property{i}' for i in range(1, 11)]
target = target_cols[tn-1]

# Prepare features (X) and target (y) for Property 1
# Assuming df_train is loaded and contains the necessary columns
if 'df' in locals():
    try:
        X = df_test[fraction_cols + property_tn ]
        y = df_test[target_cols[tn-1]]
        print("Features (X) and target (y) for BlendProperty1 prepared successfully.")

        # Predict with the EagleBlendPredictor for targets 1
        # The predict_BlendProperty1 method expects a row (DataFrame slice or similar)
        # Since X is a DataFrame, we can iterate through rows or pass the DataFrame if the method supports it.
        # Based on the current predict_BlendProperty1 signature, it seems to expect a single row or array-like.
        # Let's assume for now that predict_BlendProperty1 can handle a DataFrame directly,
        # or it processes row by row internally. If not, this part needs adjustment.
        # Assuming predict_BlendProperty1 takes the feature DataFrame directly:
        start =time.time()
        predictions_prop1 = predictors[tn-1](X)
        end = time.time()
        print("\nPrediction time:",end - start)
        print("\nPredictions for BlendProperty1 generated successfully.")
        print("First 5 predictions:", predictions_prop1[:5])

    except KeyError as e:
        print(f"Error: Missing column in DataFrame: {e}")
    except Exception as e:
        print(f"An error occurred during feature preparation or prediction: {e}")
else:
    print("df_train not found. Please ensure the training data was loaded successfully.")

In [ ]:
# Evaluate the predictions for BlendProperty1 using MAPE
try:
    mape_prop1 = mean_absolute_percentage_error(y, predictions_prop1)
    print(f"\nMean Absolute Percentage Error (MAPE) for BlendProperty1: {mape_prop1:.4f}")

except Exception as e:
    print(f"An error occurred during evaluation: {e}")

In [ ]:
tn